# <center>Description of the Project</center>

In [3]:
from IPython.display import Image

## 1.Topic

The main content of our project is about a geometric gradient descent algorithm——GeoD.  

GeoD  is a new method for unconstrained optimization of a smooth and strongly convex function. As argued and proved in "A geometric alternative to Nesterov’s accelerated gradient descent" (__Sébastien Bubeck, Yin Tat Lee, Mohit Singh__, 2015), GeoD could attain the optimal rate of convergence of Nesterov’s accelerated gradient descent. One advantage of this geometric algorithm is that it has a much simpler interpretation than Nesterov's accelerated algorithm.

## 2.Therotical Advantages of GeoD

Considering a $\beta$-smooth and $\alpha$-strongly function $f:~\mathbb{R}^n\rightarrow \mathbb{R}$, we have 
$$
\forall y\in \mathbb{R}^{n},~f(y)\geq f(x)+\nabla f(x)^\top (y-x) + \frac{\alpha}{2}|y-x|^2, \\
f(x^\top)\leq f(x)-\frac{1}{2\beta}|\nabla f(x)|^2
$$
For a given guarantee $R_0>0$ on the distance from some points $x_0$ to the optimum that is $x^*\in B(x_0, R_0^2)$, we can derive that the solution is in a ball:
$$
x^* \in B\left(x_0^{++}, \frac{|\nabla f(x_0)|^2}{\alpha^2}\left(1-\frac{1}{\kappa}\right)- \frac{2}{\alpha}\left(f(x_{0}^+)-f(x^*)\right)\right)
$$
where $x_0^+ = x - \frac{1}{\beta}\nabla f(x_0)$, $x_0^{++}=x-\frac{1}{\alpha}\nabla f(x_))$, $\kappa = \frac{\beta}{\alpha}$. This ball shrinks by a factor of $1-\frac{1}{\kappa}$.

Also, we are given a guarantee $R_0 >0$ such that $x^* \in B\left(x_0, R_0^2-\frac{2}{\alpha}\left(f(y)-f(x^*)\right)\right)$, where $f(x_0)=f(y)$, and therefore can obtain another ball
$$
x^* \in B\left(x_0, R_0^2-\frac{|\nabla f(x_))|^2}{\alpha^2 \kappa}- \frac{2}{\alpha}\left(f(x_0^+)-f(x^*)\right) \right)
$$
By the Lemma 1 in **[1]**, we have a new ball enclosing the intersection of the above shrunk balls such that 
$$
x^* \in B\left(x_1^{'}, R_0^2\left(1-\frac{1}{\sqrt{\kappa}}\right)-\frac{2}{\alpha}\left(f(x_0^+ - f(x^*))\right)\right)
$$
where $f(x_1^{'}\leq f(x_0^+))$. It gives us an acceleration in shrinking of the radius which is same as Nesterov's accelerated gradient descent.

## 3.Analysis Plan

1.We implement GeoD algorithm following the pseudocode shown in the paper:

<img src="A1.png" width="500" height="300">

<img src="A2.png" width="500" height="300">

2.To confirm the gradient descent rate of GeoD, we will compare GeoD's performance with other optimizers such as SGD, Adam, Nesterov's method. The comparisons will be condcuted under several different loss functions in order to make the conclusion more solid and persuasive. The necessity of the strong convexity of loss function will also be checked.

3.In order to figure out how the algorithm of "Line Search" functions in GeoD, we will make modifications to it and compare the Geo performances. In this part, we will apply the methodology of "control variables" to compare the importance of 2 separate parameters in "Line Search" algoreithm: length of line segment we search from & number of points we search from.

4.Also, we will record the time cost for each optimizer and each modification of GeoD algorithm. Combined each algorithm's time cost and performance, we can make a more comprehensive conclusion. 

## 4.Main Body

## A. Preparation Work before Experiments: Framework Building

#### 1.Loss Function Class

In our whole experiment, we consider four loss function class under two scenario: **linear regression scenario** and **support vector machine classification scenario**. As it is mentioned in the paper, GeoD algorithm requires strong convexity for the loss function. Some loss functions class we consider are strong convex while some are just convex, so we are able to observe if non-strong convex function also performs well for GeoD algorithm from the simulation result.

For linear regression, we consider mean square error as our loss function. For support vector machine, we consider hinge, quadratic hinge and smooth hinge as our loss functions. For each of the four loss function class, we also add an option for L2 regularization.

#### 2.Optimizers Class

In our whole experiment, we consider five gradient descent algorithms: **Stochastic Gradient Descent**, **Classical Momentum**, **Nesterov's Accelerated Gradient Descent**, **Adam**, and **GeoD**. We implemented the GeoD method. For other four methods, we referred the code in reference[2].

(Tips: In GeoD, the strong convexity parameter "__alpha__" is needed during each iteration. We calculate the minimal eigenvalue of $2X^TX$ to derive value of "alpha". The result is a little bit more than 10, thus we taking alpha as 10. However, for another 3 classification hinge loss functions, since they are __not strongly convex__, we can not calculate a positive alpha accordingly. In order to make sure GeoD works in these non-strongly convex functions, we make alpha __defaulted to be the same value 10__.)

## B. Experiment I --- Comparisons of GeoD with other Optimizers

1. Simulate data separately for the linear regression loss and 3 classification losses.  

2. Record the iterated measurement for each algorithm under each loss function and collectively show them in plot:

   a. For linear regression loss function, we take MSE as the evaluation of algorithm performance which is set as y-axis label in plot.  
   b. For 3 hinge loss functions, we take both prediction accuracy and parameter error as the evaluations of alogirithm performance        which are set as y-axis label in plot separately.  
  
3. Derive conclusion on GeoD's performance based on the plot.

### Conclusions for Experiment I:

Based on the results from Experiment I, especially during the very first several iterations, GeoD has better performance than other optimizers including the Nesterov's accelerated algorithm. However, GeoD does not have an overwhelming performance regarding the final perfromance after the same number of iterations as other optimizers.

## C. Experiment II --- Dip into the core algorithm in GeoD: Line Search

##### Motivatioon for Experiment II

Based on  our early findings in Experimant I, GeoD tend to attain a good iteration result in a very few interation times which makes it stand out from other optimizers including Nesterov's accelerated algorithm. This phenomena occurs under each of the loss functions given any initilized data.  

This phenomena encourages us to confirm the effect of the core algorithm in GeoD's each iteration process: Line Search algorithm.  
Our method is to reduce the effect of the searching algorithm by cutting down important parameters within Line Search algorithm and to make comparisons between them.

(Tips: In addition to the work below, we have tried to modify the line search algorithm to simply return the __middle point__ of the 2 take-in points, the result turns out to be a pattern of __divergence__ which means GeoD on longer works.)

##### Methodology in Experiment II

Generally there are 2 factors that are relevant to Line Search's effect:  
__1.Length of the line segment we search from__  
__2.The number of points from the line segment we take__  

Consequently, we make modifications to the 2 dimensions above separately in the Line Search algorithm which does not affect the whole structure and mechanism of GoeD algorithm. Then we focus on the comparisons of the 4 different GeoD versions under different loss functions:  

__Geo_full__: Full length of line segment and 300 points taken from the line segment.  
__Geo_quarter_division__: Full length of line segment and 4 points taken from the line segment.  
__Geo_one-third__: One-third the length of line segment and 100 points taken from the line segment.  
__Geo_one-third_quarter_division__: One-third the length of line segment and 4 points taken from the line segment. 

### Conclusions for Experiment II:

Under the smooth and strongly convex loss function of linear regression, we obtain the following findings:  

1.When the length of the line segment is fixed to be full, if we reduce the number of points from 300 to 4, the speed of Line Search is greatly reduced in the beginning iterations.  

2.When the length of the line segment is fixed to be one-third, if we reduce the number of points from 300 to 4, the speed of Line Search is greatly reduced in the beginning iterations.  

3.When the number of points taken from the line segment is fixed to be 300, if we reduce the length of line segment from full to one-third, the effect of Line Search and the performance of GeoD algorithm generally have no difference.  

4.When the number of points taken from the line segment is fixed to be 4, if we reduce the length of line segment from full to one-third, the effect of Line Search and the performance of GeoD algorithm are greatly influenced.

__The above findings reveal that the number of the points we take from the line segment for finding the local minimum is an important guarantee for the effect of Line Search. When the number of points is ensured to be large enough, the modifications to be length of line segment we search from will not make obvious difference.__

__Also, from Experiment II, we can conclude that it is the Line Search algorithm that makes the performance of GeoD so outstanding among other optimizers especially during the very first several iterations.__

## D. From another Angle: Time Cost and corresponding Inspiration

##### Motivation

Now we have concluded that the outstanding performance of GeoD during the early stage mainly result from the aceelerating effect of "Line Search". However, up to now, we simply analyzed the advantages of GeoD and the reason behind.  

The fact is that, pretty intuitively, the GeoD algorithm is very time-consuming. In this part, we make a summary of the time cost of different optimizers and different versions of GoeD we applied.

##### Table of different optimizers' time cost (cumulative time among entire 300 iertations) (unit: second)

| Optimizers/Loss | Quadratic_linear | Hinge | Quadratic_hinge | Smooth_hinge |
| --- | --- | --- | --- | --- |
| Classical Momentem |<0.01 | 0.594 | 0.813 | 0.813 |
| Nesterov | <0.01 | 0.656 | 0.578 | 0.938 |
| Adam | 0.015625 | 0.797 | 0.672 | 1.703 |
| Geo_full | 9.6875 | 349.4 | 499.3 | 540.3 |  

__Comparisons between GeoD and other optimizers:__  

The first thing not demonstrated from the first table is that the real time efficiency of GeoD is higher than that shown in the table because GeoD often attains the converged level in a much shorted time than other optimizers we list here.  

Despite the faster speed of achieving the optimal result, it is a definite conclusion that GeoD cost much more time than other optimizers. And, as discussed in former experiment, it results from the time-consuming algorithm "Line Search". So if we combine the GeoD's fasters speed during the very early stage and its much greater running time, we think the advantages of GeoD no longer stand out. 
  
However, if the machine's computing ability is strong enough, we say GeoD is still a very good choice especially during the very first several iterations.

##### Table of different GeoD modifications' time cost (cumulative time among entire 300 iertations) (unit: second)

| Optimizers/Loss | Quadratic_linear | Hinge | Quadratic_hinge | Smooth_hinge |
| --- | --- | --- | --- | --- |
| Geo_full | 99.47 | 145.4 | 216.7 | 270.7 |
| Geo_quarter | 2.125 | 3.672 | 5.109 | 3.594 |
| Geo_one-third | 31.73 | 50.20 | 83.14 | 84.89 |
| Geo_one-third_quarter | 2.375 | 3.156 | 4.859 | 3.969 |

__Comparisons between GeoD's different modifications:__  

When we greatly reduce the number of points we take for line searching, GeoD on longer costs so much time. However, this will make GeoD's performance wrosen. Based on the plot in the former experiment, when we reduce it to 4 points per iteration, the finally iterated result has a obvious gap from that of full GeoD.  

When we reduce the length of the line segment by two-thirds, the running time will decrease accordingly. Based on the plot in the former experiment, when the line segment length is reduced to one-third of the orginal one, the performance of GeoD is not greatly affected. From this angle, we can propose a constructive suggestion for the improvement of GeoD that __the area of Line Search could be shortend__ to drastically save the running time while the performance of GeoD generally will remain the same. 

## 5.Conclusions

__1.GeoD's performance in strongly convex loss function__

As the main purpose of our project, we compared GeoD's performance with other optimizers under quadratic_linear loss functions. Based on the plot. we find that GeoD tends to converge to the minimum loss value in very few iteration times which makes it stand out from other optimizers. However, the cost of the fast iteration rate is huge amount of running time. The huge time cost mainly comes from the core algorithm in GeoD---Line Search. Generally speaking, we have confirmed the iteration rate of GoeD. 

__2.Strong and non-strong convexity of loss functions__

In our experiments, we collected results from the strongly convex loss function of linear regresion as well as several non-strongly convex classification loss functions {hinge, quadratic_hinge, smooth_hinge}. we can see that GeoD overwhelms other optimizers during the early stage of iterations under strongly convex loss functions. However, for non-strongly ones, it indicates that this kind of advantage is no longer an obvious pattern. Although GeoD still converges in these non-strongly convex losses, a faster speed towards the end is not a guarantee. So are the final converged levels of both parameter error and prediction accuracy. The possible reason is that the effect of Line Search is influenced in non-strongly convex losses.

__3.Findings about linear search__

Now we are aware of the accelerating effect of Line Search towards the minimum loss's direction. Then we use control variable methodology to investigate into how the paramaters inside Line Search algorithm influence its accelerating effeict. Two features of Line Search algorithm are analyzed: 1.length of the line segment 2.number of points we search from the line segment.  

The result shows that even if we reduce the length of the line segment by two-thirds, as long as we keep the number of points we search from is large enough, the accelerating effect of Line Search will not be affected much. However, for some of the results we obtained, Geo_one-third_quarter_division sometimes could be better. This happens because of the randomness that the possibility exists, one of the quarter division points is close enough to the local minimum value.  

An inspirement we obtain from the experiment on Line Search is that we can greatly increase the time efficiency of GeoD by appropriately cutting the range of the straight line we search from. 

## 6.Reference

[1] Sébastien Bubeck, Yin Tat Lee, Mohit Singh. 2015. "A geometric alternative to Nesterov’s accelerated gradient descent".  
[2] https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/neural_network/_stochastic_optimizers.py  
[3] https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/optimizers.py